In [1]:
import pandas as pd
import numpy as np
import os
from google.colab import drive
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from mlxtend.frequent_patterns import apriori, association_rules
import joblib
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
import joblib

# Mount Google Drive
drive.mount('/content/drive')

# Path of folder
folder_path = '/content/drive/MyDrive/Colab Notebooks/2024_08_09/'

# Get list of all CSV files in the folder
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty list to hold the dataframes
dfs = []

# Loop through the CSV files and read each one into a dataframe
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all dataframes into a single dataframe
data = pd.concat(dfs, ignore_index=True)


Mounted at /content/drive


<ipython-input-1-1f7102db6cd6>:30: DtypeWarning: Columns (4,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


In [2]:
import pandas as pd
import numpy as np

# Assuming data is already loaded
# Convert the 'Occurred On' to datetime
data['First Occurred On'] = pd.to_datetime(data['First Occurred On'])

# Filter data for the specific Alarm Name
target_alarm = 'RF Unit TX Channel Gain Out of Range'
filtered_data = data[data['Alarm Name'] == target_alarm]

# Get the unique Site IDs
site_ids = filtered_data['Site ID'].unique()

# Print the Site IDs
print("Site IDs with the target alarm:", site_ids)

# Filter the initial dataset to get all alarms for the identified Site IDs
alarms_for_site_ids = data[data['Site ID'].isin(site_ids)]

# Print the filtered alarms
print("Alarms for the identified Site IDs:")
print(alarms_for_site_ids.head())

# Sort by site and date
df = alarms_for_site_ids.sort_values(by=['Site ID', 'First Occurred On'])

# Filter the target alarm data
df['Target'] = (df['Alarm Name'] == target_alarm).astype(int)

# Define a time window for examining past alarms (e.g., 7 days)
time_window = 14

# Create a relative day index for each alarm
df['Relative Day Index'] = df.groupby('Site ID')['First Occurred On'].transform(lambda x: (x - x.max()).dt.days)

# Pivot the data to get the binary occurrence matrix for alarms
pivot_df = df.pivot_table(index=['Site ID', 'Relative Day Index'],
                          columns='Alarm Name',
                          aggfunc='size', fill_value=0)

# Binary encode the presence of alarms
pivot_df = (pivot_df > 0).astype(int)
pivot_df.head()


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Site IDs with the target alarm: ['BA5020' 'AM0053' 'GM0694' 'CM0880' 'GM0067' 'BA0170' 'CM0204' 'CM0445'
 'CM0981' 'BA0002' 'KU0124' 'KU0045' 'AM0082' 'GM0033' 'GM0131' 'JA0014'
 'CM1304' 'VA0008' 'KI0099' 'KU0553' 'GM0271' 'CM0363' 'CM0891' 'PU0009'
 'JA0129' 'TR0110' 'CM2144' 'CM0047' 'KL0116' 'GM0230' 'GM0097' 'KU0164'
 'GM0265' 'CM0342' 'AM0212' 'PU0062' 'PU0031' 'AM0118' 'CM2008' 'GM0405'
 'MR0031' 'NU0241' 'CM0192' 'PU0022' 'GM0474' 'CM0185' 'CM0862' 'AM0033'
 'AM0247' 'PU0317' 'PO5002' 'GM0115' 'PU0075' 'CM0121' 'KE0026' 'CM0923'
 'GA0006' 'JA0058' 'GM0308' 'MA0036' 'AM0037' 'CM0345']
Alarms for the identified Site IDs:
         Last Occurred On                     Alarm Source Alarm Severity  \
1275  2024-08-09 15:02:45              Bakkiella-AM0033-DG          Major   
1302  2024-08-09 15:02:40  MC2_Kumbalamadu_Lamp-BA5020-DHP          Major   
3045  2024-08-09 14:30:00   ALVAI_EAST-CETR-OFN18-R1-N1-01       Critical   
3392  2024-08-09 14:23:32  MC2_Kumbalamadu_Lamp-BA5020-DH

Alarm Name                  (PTP)Time synchronization faulty--2501553  \
Site ID Relative Day Index                                              
AM0033  -31.0                                                       0   
        -30.0                                                       0   
        -29.0                                                       0   
        -28.0                                                       0   
        -27.0                                                       0   

Alarm Name                  (Sync E)The state of clock source is failed--2600422  \
Site ID Relative Day Index                                                         
AM0033  -31.0                                                               0      
        -30.0                                                               0      
        -29.0                                                               0      
        -28.0                                                               0      
        -27.0                                                               0      

Alarm Name                  (Sync E)The work mode of system clock is not in locked mode--2600420  \
Site ID Relative Day Index                                                                         
AM0033  -31.0                                                               0                      
        -30.0                                                               0                      
        -29.0                                                               0                      
        -28.0                                                               0                      
        -27.0                                                               0                      

Alarm Name                  AM_DOWNSHIFT  Ambient Temperature Unacceptable  \
Site ID Relative Day Index                                                   
AM0033  -31.0                          0                                 0   
        -30.0                          0                                 0   
        -29.0                          0                                 0   
        -28.0                          0                                 0   
        -27.0                          0                                 0   

Alarm Name                  BBU Board Maintenance Link Failure  \
Site ID Relative Day Index                                       
AM0033  -31.0                                                0   
        -30.0                                                0   
        -29.0                                                0   
        -28.0                                                0   
        -27.0                                                0   

Alarm Name                  BBU CPRI Interface Error  \
Site ID Relative Day Index                             
AM0033  -31.0                                      0   
        -30.0                                      0   
        -29.0                                      0   
        -28.0                                      0   
        -27.0                                      0   

Alarm Name                  BBU CPRI Optical Module Fault  \
Site ID Relative Day Index                                  
AM0033  -31.0                                           0   
        -30.0                                           0   
        -29.0                                           0   
        -28.0                                           0   
        -27.0                                           0   

Alarm Name                  BBU CPRI Optical Module or Electrical Port Not Ready  \
Site ID Relative Day Index                                                         
AM0033  -31.0                                                               0      
        -30.0                                                               0      
        -29.0                  

In [3]:
from mlxtend.frequent_patterns import apriori, association_rules

# Apply the Apriori algorithm with a min_support
frequent_itemsets = apriori(pivot_df, min_support=0.005, use_colnames=True)

# Generate association rules with the target alarm as the consequent, using confidence as the metric
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)  # Adjust min_threshold as needed
target_rules = rules[rules['consequents'] == frozenset([target_alarm])]

# Print all target rules to see the results
print("All Target Rules:\n", target_rules)

# Create a DataFrame to store the antecedent and their counts
antecedent_counts = target_rules['antecedents'].apply(lambda x: list(x)[0] if x else None).value_counts()

# Extract the top 5 common alarms
top_5_common_alarms = antecedent_counts.head().index.tolist()

print("Top 5 Common Alarms that frequently precede the target alarm:", top_5_common_alarms)
print("Number of frequent itemsets:", len(frequent_itemsets))


top_5_common_alarms[:10]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


All Target Rules:
                                             antecedents  \
31                                 (Carrier No Traffic)   
771                    (Cell Out of Service, Link Down)   
818           (Cell Out of Service, Physical Port Down)   
3418          (Cell Out of Service, Link Down, ETH_LOS)   
3453  (Cell Out of Service, Physical Port Down, ETH_...   
3645  (Cell Out of Service, Physical Port Down, Link...   
7880  (Cell Out of Service, Physical Port Down, Link...   

                                 consequents  antecedent support  \
31    (RF Unit TX Channel Gain Out of Range)            0.013008   
771   (RF Unit TX Channel Gain Out of Range)            0.011382   
818   (RF Unit TX Channel Gain Out of Range)            0.011382   
3418  (RF Unit TX Channel Gain Out of Range)            0.011382   
3453  (RF Unit TX Channel Gain Out of Range)            0.011382   
3645  (RF Unit TX Channel Gain Out of Range)            0.011382   
7880  (RF Unit TX Channel Gain O

['Cell Out of Service', 'Carrier No Traffic']

In [9]:
target_rules.head(20)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
31,(Carrier No Traffic),(RF Unit TX Channel Gain Out of Range),0.013008,0.206504,0.006504,0.5,2.42126,0.003818,1.586992,0.594728
771,"(Cell Out of Service, Link Down)",(RF Unit TX Channel Gain Out of Range),0.011382,0.206504,0.005691,0.5,2.42126,0.003341,1.586992,0.593750
818,"(Cell Out of Service, Physical Port Down)",(RF Unit TX Channel Gain Out of Range),0.011382,0.206504,0.005691,0.5,2.42126,0.003341,1.586992,0.593750
3418,"(Cell Out of Service, Link Down, ETH_LOS)",(RF Unit TX Channel Gain Out of Range),0.011382,0.206504,0.005691,0.5,2.42126,0.003341,1.586992,0.593750
3453,"(Cell Out of Service, Physical Port Down, ETH_...",(RF Unit TX Channel Gain Out of Range),0.011382,0.206504,0.005691,0.5,2.42126,0.003341,1.586992,0.593750
3645,"(Cell Out of Service, Physical Port Down, Link...",(RF Unit TX Channel Gain Out of Range),0.011382,0.206504,0.005691,0.5,2.42126,0.003341,1.586992,0.593750
7880,"(Cell Out of Service, Physical Port Down, Link...",(RF Unit TX Channel Gain Out of Range),0.011382,0.206504,0.005691,0.5,2.42126,0.003341,1.586992,0.593750


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['antecedent support'].plot(kind='hist', bins=20, title='antecedent support')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_19['antecedents'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_19, x='support', y='antecedents', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_20['antecedents'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_20, x='leverage', y='antecedents', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_ex

In [4]:

# Commenting out original filtering based on common alarms and target
# df_common = df[df['Alarm Name'].isin(common_alarms | {target_alarm})]


common_alarms = top_5_common_alarms
# Create lag features for each common alarm
# Ensure common_alarms is not empty before proceeding
if common_alarms:
    for alarm in common_alarms:
        # Create lag features only for common alarms that are identified
        # Use the original 'Alarm Name' column to check for the alarm
        df[alarm + '_lag'] = df.groupby('Site ID')['Alarm Name'].transform(lambda x: (x == alarm).shift(1).fillna(0))

# Commenting out original target creation, changing the way to calculate the target
# Create a target variable indicating whether the target alarm occurs within the next 7 days
df['Target Next 7 Days'] = df.groupby('Site ID')['Target'].shift(-7).fillna(0)

# Drop rows with missing values in the target variable
df.dropna(subset=['Target Next 7 Days'], inplace=True)

# Define features and target, ensure features is not empty
features = [alarm + '_lag' for alarm in common_alarms]
if features:  # Check if features list is not empty
    X = df[features]
    y = df['Target Next 7 Days']

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Normalize the features
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    # Train a Random Forest model
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    model.fit(X_train_scaled, y_train)

    # Save the model and scaler
    joblib.dump(model, 'rf_model.pkl')
    joblib.dump(scaler, 'scaler.pkl')
else:
    print("No common alarms found. Cannot proceed with model training.")





/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [5]:


# Predict on the test set
y_pred = model.predict(X_test_scaled)
y_pred_prob = model.predict_proba(X_test_scaled)[:, 1]

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("AUC-ROC Score:", roc_auc_score(y_test, y_pred_prob))



Accuracy: 0.9322101885278781
Classification Report:
               precision    recall  f1-score   support

         0.0       0.93      1.00      0.96      4648
         1.0       0.00      0.00      0.00       338

    accuracy                           0.93      4986
   macro avg       0.47      0.50      0.48      4986
weighted avg       0.87      0.93      0.90      4986

AUC-ROC Score: 0.5140207915346933


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/

In [6]:
import os
import pandas as pd
import joblib

# Load the model and scaler
model = joblib.load('rf_model.pkl')
scaler = joblib.load('scaler.pkl')

# Path of the folder
folder_path = '/content/drive/MyDrive/Colab Notebooks/2024_08_09/'

# Get list of all CSV files in the folder
csv_files = [os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.csv')]

# Initialize an empty list to hold the dataframes
dfs = []

# Loop through the CSV files and read each one into a dataframe
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

# Concatenate all dataframes into a single dataframe
new_data = pd.concat(dfs, ignore_index=True)

# Convert 'First Occurred On' to datetime
new_data['First Occurred On'] = pd.to_datetime(new_data['First Occurred On'])

# Ensure that the columns in the new data match the features used during model training
columns_in_model = scaler.feature_names_in_
available_features = [feature for feature in columns_in_model if feature in new_data.columns]

# Check if there are any available features
if not available_features:
    print("No features from the model are present in the new dataset. Prediction cannot proceed.")
else:
    # If some features are missing, inform the user
    if len(available_features) < len(columns_in_model):
        print(f"Some features are missing. Proceeding with available features: {available_features}")

    # Scale the new data based on the available features
    X_new_scaled = scaler.transform(new_data[available_features])

    # Predict the probability of the target alarm occurring within the next 7 days
    pred_prob = model.predict_proba(X_new_scaled)[:, 1]

    # Add the prediction probability to the new data
    new_data['Prediction Probability'] = pred_prob

    # Extract and print unique Site IDs
    unique_site_ids = new_data['Site ID'].unique()
    print("Unique Site IDs:")
    print(unique_site_ids)

    # Optionally, you can save the predictions and unique Site IDs to a new CSV file
    output_file = os.path.join(folder_path, 'predictions_with_unique_site_ids.csv')
    new_data[['Site ID', 'Prediction Probability']].drop_duplicates().to_csv(output_file, index=False)
    print(f"Predictions and unique Site IDs saved to {output_file}")


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-6-f963c3da8c5b>:20: DtypeWarning: Columns (4,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


No features from the model are present in the new dataset. Prediction cannot proceed.


In [7]:
# Print the features used in the model
features_in_model = scaler.feature_names_in_
print("Features used in the model:")
print(features_in_model)

Features used in the model:
['Cell Out of Service_lag' 'Carrier No Traffic_lag']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
